# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [8]:
import pandas as pd
import numpy as np
import json

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [2]:
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip', low_memory=False)

loans


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [3]:
# Crear un DataFrame con los nombres de las columnas y sus tipos de datos
column_types = pd.DataFrame({
    'column_name': loans.columns,
    'type': loans.dtypes.values
})

# Guardar el DataFrame de tipos de columna en un archivo CSV
#column_types.to_csv('column_types.csv', index=False)

# Mostrar el DataFrame de tipos de columna
column_types

,column_name,type
0,Unnamed: 0,int64
1,id,int64
2,member_id,float64
3,loan_amnt,float64
4,funded_amnt,float64
...,...,...
146,settlement_status,object
147,settlement_date,object
148,settlement_amount,float64
149,settlement_percentage,float64


## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [4]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [6]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [7]:
# Codigo guardar
datos_dict.to_pickle("datos_dict.pkl")

In [8]:
# Codigo para cargar
datos_dict_cargado = pd.read_pickle("datos_dict.pkl")

## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [ ]:
# Columnas de tipo entero
int_columns = {
    'id': 'int',
    'member_id': 'int',
    'loan_amnt': 'int',
    'funded_amnt': 'int',
    'funded_amnt_inv': 'int',
    'delinq_2yrs': 'int',
    'fico_range_low': 'int',
    'fico_range_high': 'int',
    'inq_last_6mths': 'int',
    'mths_since_last_delinq': 'int',
    'mths_since_last_record': 'int',
    'open_acc': 'int',
    'pub_rec': 'int',
    'total_acc': 'int',
    'last_fico_range_high': 'int',
    'last_fico_range_low': 'int',
    'collections_12_mths_ex_med': 'int',
    'mths_since_last_major_derog': 'int',
    'policy_code': 'int',
    'acc_now_delinq': 'int',
    'tot_coll_amt': 'int',
    'tot_cur_bal': 'int',
    'open_acc_6m': 'int',
    'open_act_il': 'int',
    'open_il_12m': 'int',
    'open_il_24m': 'int',
    'open_rv_12m': 'int',
    'open_rv_24m': 'int',
    'inq_fi': 'int',
    'total_cu_tl': 'int',
    'inq_last_12m': 'int',
    'acc_open_past_24mths': 'int',
    'avg_cur_bal': 'int',
    'bc_open_to_buy': 'int',
    'chargeoff_within_12_mths': 'int',
    'delinq_amnt': 'int',
    'mo_sin_old_il_acct': 'int',
    'mo_sin_old_rev_tl_op': 'int',
    'mo_sin_rcnt_rev_tl_op': 'int',
    'mo_sin_rcnt_tl': 'int',
    'mort_acc': 'int',
    'num_accts_ever_120_pd': 'int',
    'num_actv_bc_tl': 'int',
    'num_actv_rev_tl': 'int',
    'num_bc_sats': 'int',
    'num_bc_tl': 'int',
    'num_il_tl': 'int',
    'num_op_rev_tl': 'int',
    'num_rev_accts': 'int',
    'num_rev_tl_bal_gt_0': 'int',
    'num_sats': 'int',
    'num_tl_120dpd_2m': 'int',
    'num_tl_30dpd': 'int',
    'num_tl_90g_dpd_24m': 'int',
    'num_tl_op_past_12m': 'int',
    'percent_bc_gt_75': 'float',
    'pub_rec_bankruptcies': 'int',
    'tax_liens': 'int',
    'tot_hi_cred_lim': 'int',
    'total_bal_ex_mort': 'int',
    'total_bc_limit': 'int',
    'total_il_high_credit_limit': 'int',
    'sec_app_fico_range_low': 'int',
    'sec_app_fico_range_high': 'int',
    'sec_app_inq_last_6mths': 'int',
    'sec_app_mort_acc': 'int',
    'sec_app_open_acc': 'int',
    'sec_app_open_act_il': 'int',
    'sec_app_num_rev_accts': 'int',
    'sec_app_chargeoff_within_12_mths': 'int',
    'sec_app_collections_12_mths_ex_med': 'int',
    'sec_app_mths_since_last_major_derog': 'int',
    'deferral_term': 'int',
    'hardship_dpd': 'int',
    'settlement_term': 'int'
}

# Columnas de tipo flotante
float_columns = {
    'int_rate': 'float',
    'installment': 'float',
    'annual_inc': 'float',
    'dti': 'float',
    'revol_bal': 'int',
    'revol_util': 'float',
    'out_prncp': 'float',
    'out_prncp_inv': 'float',
    'total_pymnt': 'float',
    'total_pymnt_inv': 'float',
    'total_rec_prncp': 'float',
    'total_rec_int': 'float',
    'total_rec_late_fee': 'float',
    'recoveries': 'float',
    'collection_recovery_fee': 'float',
    'last_pymnt_amnt': 'float',
    'annual_inc_joint': 'float',
    'dti_joint': 'float',
    'total_bal_il': 'float',
    'il_util': 'float',
    'max_bal_bc': 'float',
    'all_util': 'float',
    'bc_util': 'float',
    'pct_tl_nvr_dlq': 'float',
    'total_rev_hi_lim': 'int',
    'settlement_amount': 'float',
    'settlement_percentage': 'float',
    'hardship_amount': 'float',
    'orig_projected_additional_accrued_interest': 'float',
    'hardship_payoff_balance_amount': 'float',
    'hardship_last_payment_amount': 'float'
}

# Columnas de tipo fecha
date_columns = {
    'issue_d': 'datetime64[ns]',
    'earliest_cr_line': 'datetime64[ns]',
    'last_pymnt_d': 'datetime64[ns]',
    'next_pymnt_d': 'datetime64[ns]',
    'last_credit_pull_d': 'datetime64[ns]',
    'hardship_start_date': 'datetime64[ns]',
    'hardship_end_date': 'datetime64[ns]',
    'payment_plan_start_date': 'datetime64[ns]',
    'debt_settlement_flag_date': 'datetime64[ns]',
    'settlement_date': 'datetime64[ns]',
    'sec_app_earliest_cr_line': 'datetime64[ns]'
}

# Columnas de tipo cadena y categoría
string_columns = {
    'term': 'string',
    'emp_title': 'string',
    'emp_length': 'string',
    'home_ownership': 'string',
    'verification_status': 'string',
    'loan_status': 'string',
    'pymnt_plan': 'category',
    'desc': 'string',
    'purpose': 'string',
    'title': 'string',
    'zip_code': 'string',
    'addr_state': 'category',
    'initial_list_status': 'category',
    'application_type': 'string',
    'verification_status_joint': 'string',
    'hardship_flag': 'category',
    'hardship_type': 'string',
    'hardship_reason': 'string',
    'hardship_status': 'string',
    'hardship_loan_status': 'string',
    'disbursement_method': 'string',
    'debt_settlement_flag': 'category',
    'settlement_status': 'string',
}

# Consolidar todas las categorías en un solo diccionario
new_types = {**int_columns, **float_columns, **date_columns, **string_columns}

# Cambiar los tipos de datos
for col in loans.columns:
    if col in new_types and new_types[col] == 'int':
        loans[col] = loans[col].astype('Int64')  # Tipo que permite NaN
    elif col in new_types:
        loans[col] = loans[col].astype(new_types[col])


In [10]:
# Crear un DataFrame con los nombres de las columnas y sus tipos de datos
column_types = pd.DataFrame({
    'column_name': loans.columns,
    'type': loans.dtypes.values
})

# Guardar el DataFrame de tipos de columna en un archivo CSV
column_types.to_csv('column_New_types.csv', index=False)

# Mostrar el DataFrame de tipos de columna
column_types


,column_name,type
0,Unnamed: 0,int64
1,id,Int64
2,member_id,Int64
3,loan_amnt,Int64
4,funded_amnt,Int64
...,...,...
146,settlement_status,string[python]
147,settlement_date,datetime64[ns]
148,settlement_amount,float64
149,settlement_percentage,float64


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [10]:
#primero quitamos todas las columnas que solo contengan Nan porque son columnas que no aportan nada de información y ocupan espacio
def drop_nan_only_columns(df):
    """
    Drops columns that contain only NaN values.

    Parameters:
    df (pd.DataFrame): The DataFrame from which to drop NaN-only columns.

    Returns:
    pd.DataFrame: A DataFrame with NaN-only columns removed.
    """
    # Drop columns where all values are NaN
    df_cleaned = df.dropna(axis=1, how='all')
    return df_cleaned

# Example usage
# Assuming `loans` is your DataFrame
loans_cleaned = drop_nan_only_columns(loans)


# Function to handle numeric column imputation
def imputar_numerico(df, column):
    # Use mean or median based on skewness
    if abs(df[column].skew()) < 1:
        value = df[column].mean()
        strategy = 'mean'
    else:
        value = df[column].median()
        strategy = 'median'
    df[column].fillna(value)
    return {"columna":column,"estrategia": strategy, "valor": value}

def imputar_categorico(df, column):
    # Use a placeholder string for missing values
    value = "missing"
    df[column].fillna(value)
    return { "estrategia": "identificador", "valor": value}


# Function to handle date column imputation
def imputar_fecha(df, column):
    # Use a placeholder date for missing values
    value = pd.Timestamp("1900-01-01")
    df[column].fillna(value)
    return {"estrategia": "placeholder_date", "valor": str(value)}

# Function to handle boolean column imputation
def imputar_booleano(df, column):
    # Use False as the default for missing values
    value = False
    df[column].fillna(value)
    return {"estrategia": "default_bool", "valor": value}

# Main function to apply imputations based on data type
def imputar_valores(df):
    imputation_strategies = {}
    for column in df.columns:
        if df[column].isna().sum() > 0:
            print("woo")
            if pd.api.types.is_numeric_dtype(df[column]):
                imputation_strategies[column] = imputar_numerico(df, column)
            elif pd.api.types.is_object_dtype(df[column]):
                imputation_strategies[column] = imputar_categorico(df, column)
            elif pd.api.types.is_datetime64_any_dtype(df[column]):
                imputation_strategies[column] = imputar_fecha(df, column)
            elif pd.api.types.is_bool_dtype(df[column]):
                imputation_strategies[column] = imputar_booleano(df, column)
        else:
            imputation_strategies[column] = {"estrategia": "sin_imputacion", "valor": None}
    return imputation_strategies

explanation_strategies = {
    "mean": "Replaces NaN values with the mean of the column for balanced data.",
    "median": "Replaces NaN values with the median of the column for skewed data.",
    "identificador": "Replaces NaN values in categorical columns with a placeholder like 'missing'.",
    "placeholder_date": "Replaces NaN values in date columns with a default date (e.g., '1900-01-01').",
    "default_bool": "Replaces NaN values in boolean columns with a default False value.",
    "sin_imputacion": "Indicates no imputation was needed because no NaN values were present."
}

imputation_strategies = imputar_valores(loans_cleaned)

woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo
woo


In [11]:
imputation_strategies

{'Unnamed: 0': {'estrategia': 'sin_imputacion', 'valor': None},
 'id': {'estrategia': 'sin_imputacion', 'valor': None},
 'loan_amnt': {'estrategia': 'sin_imputacion', 'valor': None},
 'funded_amnt': {'estrategia': 'sin_imputacion', 'valor': None},
 'funded_amnt_inv': {'estrategia': 'sin_imputacion', 'valor': None},
 'term': {'estrategia': 'sin_imputacion', 'valor': None},
 'int_rate': {'estrategia': 'sin_imputacion', 'valor': None},
 'installment': {'estrategia': 'sin_imputacion', 'valor': None},
 'grade': {'estrategia': 'sin_imputacion', 'valor': None},
 'sub_grade': {'estrategia': 'sin_imputacion', 'valor': None},
 'emp_title': {'estrategia': 'identificador', 'valor': 'missing'},
 'emp_length': {'estrategia': 'identificador', 'valor': 'missing'},
 'home_ownership': {'estrategia': 'sin_imputacion', 'valor': None},
 'annual_inc': {'estrategia': 'sin_imputacion', 'valor': None},
 'verification_status': {'estrategia': 'sin_imputacion', 'valor': None},
 'issue_d': {'estrategia': 'sin_impu

### Codigo para salvar y cargar JSONs

In [13]:
import json
# Save imputation strategies to JSON
with open("imputation_strategies.json", "w") as file:
    json.dump(imputation_strategies, file, indent=4)

# Save strategy explanations to JSON
with open("strategy_explanations.json", "w") as file:
    json.dump(explanation_strategies, file, indent=4)

def load_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Load the JSON containing imputation strategies for each column
imputation_strategies = load_json("imputation_strategies.json")
print("Imputation Strategies:\n", imputation_strategies)

# Load the JSON containing explanations of each strategy
strategy_explanations = load_json("strategy_explanations.json")
print("\nStrategy Explanations:\n", strategy_explanations)    

Imputation Strategies:
 {'Unnamed: 0': {'estrategia': 'sin_imputacion', 'valor': None}, 'id': {'estrategia': 'sin_imputacion', 'valor': None}, 'loan_amnt': {'estrategia': 'sin_imputacion', 'valor': None}, 'funded_amnt': {'estrategia': 'sin_imputacion', 'valor': None}, 'funded_amnt_inv': {'estrategia': 'sin_imputacion', 'valor': None}, 'term': {'estrategia': 'sin_imputacion', 'valor': None}, 'int_rate': {'estrategia': 'sin_imputacion', 'valor': None}, 'installment': {'estrategia': 'sin_imputacion', 'valor': None}, 'grade': {'estrategia': 'sin_imputacion', 'valor': None}, 'sub_grade': {'estrategia': 'sin_imputacion', 'valor': None}, 'emp_title': {'estrategia': 'identificador', 'valor': 'missing'}, 'emp_length': {'estrategia': 'identificador', 'valor': 'missing'}, 'home_ownership': {'estrategia': 'sin_imputacion', 'valor': None}, 'annual_inc': {'estrategia': 'sin_imputacion', 'valor': None}, 'verification_status': {'estrategia': 'sin_imputacion', 'valor': None}, 'issue_d': {'estrategia': 